In [1]:
# Importing libraries

import numpy as np
import pandas as pd
from datetime import datetime, date
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

In [42]:
# Scrap players data (all history)

players_name = ['113', 'Elyoya', "Caps", "Razork", "Yike", "Exakick", "Upset", "Humanoid", "Hylissang", "Nisqy"]
#players_name = ['Elyoya']

# Set up the Chrome WebDriver
driver = webdriver.Chrome()

# List where we will store the data
data = []

for player_name in players_name:
    url = f"https://liquipedia.net/leagueoflegends/{player_name}/Results"
    driver.get(url)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "table-responsive")))


    # Fins table element
    table = driver.find_element(By.CLASS_NAME, 'wikitable.wikitable-striped.sortable.jquery-tablesorter')
    table_body = table.find_element(By.TAG_NAME, 'tbody')

    # Step 3: Find all the tr elements excluding the ones with class "sortbottom"
    tr_elements = table_body.find_elements(By.CSS_SELECTOR, "tr:not(.sortbottom)")

    # Step 4 and Step 5: Extract data from td elements and store in a dataframe
    for tr_element in tr_elements:
        td_elements = tr_element.find_elements(By.CSS_SELECTOR, "td")
        player_data = [player_name] + [td.text for td in td_elements]
        data.append(player_data)

# Close the browser
driver.quit()

In [43]:
columns = ['Name', 'Date', 'Position', 'Tier', 'Unused', 'Tournament', 'Unused', 'Score', 'Unused', 'Prize']
df = pd.DataFrame(data, columns=columns)
desired_columns = ['Name', 'Date', 'Position', 'Tier', 'Tournament']
df = df[desired_columns]
df['Year'] = pd.to_datetime(df['Date']).dt.year

In [44]:
df

,Name,Date,Position,Tier,Tournament,Year
0,113,2023-07-03,9th,S-Tier,LEC Summer 2023,2023
1,113,2023-04-16,5th,S-Tier,LEC Spring 2023,2023
2,113,2023-02-19,6th,S-Tier,LEC Winter 2023,2023
3,113,2022-11-05,3rd - 4th,B-Tier,Coupe de France 2022,2022
4,113,2022-08-04,6th,B-Tier,LFL Summer 2022,2022
...,...,...,...,...,...,...
236,Nisqy,2016-02-28,5th - 6th,B-Tier,Lyon e-Sport 9,2016
237,Nisqy,2016-02-22,2nd,B-Tier,Underdogs Season 1,2016
238,Nisqy,2015-12-13,5th - 6th,B-Tier,ASUS Republic of Gamers 2015,2015
239,Nisqy,2015-11-01,3rd - 4th,B-Tier,ESL PGW Challenge 2015,2015


In [45]:
position_order = {
    '1st': 1,
    '1st - 2nd': 2,
    '1st - 3rd': 3,
    '2nd': 4,
    '3rd': 5,
    '3rd - 4th': 6,
    '4th': 7,
    '5th': 8,
    '5th - 6th': 9,
    '5th - 8th': 10,
    '6th': 11,
    '7th': 12,
    '7th - 8th': 13,
    '8th': 14,
    '9th': 15,
    '9th - 10th': 16,
    '9th - 12th': 17,
    '10th': 18,
    '11th - 14th': 19,
    '14th - 16th': 20,
    '17th - 18th': 21,
    '19th - 20th': 22,
    '19th - 22nd': 23,
    '59th': 24
}

df['relative_pos'] = df["Position"].map(position_order)

In [46]:
df = df[~df['Position'].isin(['W', 'L'])]
df = df[~df['Tier'].isin(['Showmatch', 'Misc'])]

In [47]:
tier_order = {
    'S-Tier': 1,
    'A-Tier': 2,
    'B-Tier': 3,
    'C-Tier': 4,
    'Qualifier': 5,
    'Monthly': 6

}
df['relative_tier'] = df['Tier'].map(tier_order)

In [48]:
df

,Name,Date,Position,Tier,Tournament,Year,relative_pos,relative_tier
0,113,2023-07-03,9th,S-Tier,LEC Summer 2023,2023,15.0,1
1,113,2023-04-16,5th,S-Tier,LEC Spring 2023,2023,8.0,1
2,113,2023-02-19,6th,S-Tier,LEC Winter 2023,2023,11.0,1
3,113,2022-11-05,3rd - 4th,B-Tier,Coupe de France 2022,2022,6.0,3
4,113,2022-08-04,6th,B-Tier,LFL Summer 2022,2022,11.0,3
...,...,...,...,...,...,...,...,...
236,Nisqy,2016-02-28,5th - 6th,B-Tier,Lyon e-Sport 9,2016,9.0,3
237,Nisqy,2016-02-22,2nd,B-Tier,Underdogs Season 1,2016,4.0,3
238,Nisqy,2015-12-13,5th - 6th,B-Tier,ASUS Republic of Gamers 2015,2015,9.0,3
239,Nisqy,2015-11-01,3rd - 4th,B-Tier,ESL PGW Challenge 2015,2015,6.0,3


In [53]:
# Save as csv
df.to_csv('C:/Users/adars/OneDrive/Escritorio/ProjecteLolShiny/Data/PlayerTrophies.csv', index = False)